## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-04-14-52-41 +0000,nyt,"Ivan Klima, Czech Novelist Who Chafed Under To...",https://www.nytimes.com/2025/10/04/world/europ...
1,2025-10-04-14-46-43 +0000,nyt,‘Czech Trump’ Andrej Babis Is Poised to Return...,https://www.nytimes.com/2025/10/03/world/europ...
2,2025-10-04-14-45-10 +0000,nyt,The Interview: Senator Alex Padilla Says His V...,https://www.nytimes.com/2025/10/04/magazine/al...
3,2025-10-04-14-39-38 +0000,nypost,Six people arrested on suspicion of terror off...,https://nypost.com/2025/10/04/world-news/6-sus...
4,2025-10-04-14-31-40 +0000,bbc,Harrods sets aside more than £60m for abuse co...,https://www.bbc.com/news/articles/cx20ky6n79eo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2332/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
9,trump,55
55,hamas,20
57,gaza,19
59,plan,14
476,shutdown,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
126,2025-10-04-00-42-44 +0000,nypost,Israel ready for ‘immediate implementation’ of...,https://nypost.com/2025/10/03/world-news/israe...,154
216,2025-10-03-20-03-20 +0000,nypost,Hamas agrees to release all Israeli hostages p...,https://nypost.com/2025/10/03/world-news/hamas...,149
7,2025-10-04-14-18-43 +0000,nypost,Hamas’ response to Trump’s Gaza peace plan win...,https://nypost.com/2025/10/04/world-news/hamas...,137
104,2025-10-04-03-10-45 +0000,nypost,"Hamas internal rift over hostages, weapons cou...",https://nypost.com/2025/10/03/world-news/hamas...,136
53,2025-10-04-10-44-52 +0000,wapo,High stakes for Gaza peace talks as Hamas acce...,https://www.washingtonpost.com/world/2025/10/0...,135


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
126,154,2025-10-04-00-42-44 +0000,nypost,Israel ready for ‘immediate implementation’ of...,https://nypost.com/2025/10/03/world-news/israe...
214,60,2025-10-03-20-27-00 +0000,wsj,The Trump administration has frozen $2.1 billi...,https://www.wsj.com/politics/policy/trump-admi...
92,52,2025-10-04-06-23-16 +0000,bbc,Sanae Takaichi set to become Japan's first fem...,https://www.bbc.com/news/articles/cx2pmy7m72lo...
237,52,2025-10-03-18-23-42 +0000,nypost,Terrorist Jihad Al-Shamie was on bail for rape...,https://nypost.com/2025/10/03/world-news/uk-te...
182,43,2025-10-03-21-35-06 +0000,latimes,Supreme Court says again Trump may cancel temp...,https://www.latimes.com/politics/story/2025-10...
17,41,2025-10-04-13-30-56 +0000,nypost,Two teen girls — one just 13 — found dead on B...,https://nypost.com/2025/10/04/us-news/two-teen...
43,40,2025-10-04-12-00-00 +0000,wsj,Many government leaders have had the chance to...,https://www.wsj.com/economy/trade/south-korea-...
224,37,2025-10-03-19-37-00 +0000,wsj,Trump ordered another lethal military strike o...,https://www.wsj.com/politics/national-security...
102,33,2025-10-04-03-28-52 +0000,nypost,50 Cent trolls longtime rival Sean ‘Diddy’ Com...,https://nypost.com/2025/10/03/entertainment/50...
242,32,2025-10-03-18-06-05 +0000,latimes,"Drizzly, windy weather ahead for Southern Cali...",https://www.latimes.com/california/story/2025-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
